In [1]:
print("dd")

dd


In [3]:
import os
import shutil
import random
from glob import glob

# 1. 경로 설정
# 원본 데이터가 있는 경로
source_root = r'C:\Dev\KorailWheel\data\train' 
source_img_dir = os.path.join(source_root, 'images')
source_lbl_dir = os.path.join(source_root, 'labels')

# 결과가 저장될 경로 (새로 생성됨)
output_root = r'C:\Dev\KorailWheel\data\split_dataset'

# 2. 비율 설정 (7 : 1.5 : 1.5)
train_ratio = 0.7
valid_ratio = 0.15
test_ratio = 0.15

# 3. 결과 디렉토리 생성 함수
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# 폴더 구조 생성: images/train, labels/train 등
for split in ['train', 'valid', 'test']:
    create_dir(os.path.join(output_root, 'images', split))
    create_dir(os.path.join(output_root, 'labels', split))

# 4. 파일 리스트 로드 및 셔플
# 이미지 확장자 (필요시 추가)
image_exts = ['.jpg', '.jpeg', '.png', '.bmp']
files = [f for f in os.listdir(source_img_dir) if os.path.splitext(f)[1].lower() in image_exts]

# 재현성을 위해 시드 고정 (선택 사항)
random.seed(42)
random.shuffle(files)

# 5. 분할 지점 계산
total_count = len(files)
train_idx = int(total_count * train_ratio)
valid_idx = train_idx + int(total_count * valid_ratio)

# 리스트 슬라이싱
train_files = files[:train_idx]
valid_files = files[train_idx:valid_idx]
test_files = files[valid_idx:]

print(f"총 파일 수: {total_count}")
print(f"Train: {len(train_files)} / Valid: {len(valid_files)} / Test: {len(test_files)}")
print("-" * 30)

# 6. 파일 복사 실행 함수
def copy_files(file_list, split_name):
    print(f"📂 {split_name} 데이터 복사 중...")
    for file_name in file_list:
        # 파일명과 확장자 분리
        name, ext = os.path.splitext(file_name)
        
        # 소스 경로
        src_img = os.path.join(source_img_dir, file_name)
        src_lbl = os.path.join(source_lbl_dir, name + '.txt')
        
        # 타겟 경로
        dst_img = os.path.join(output_root, 'images', split_name, file_name)
        dst_lbl = os.path.join(output_root, 'labels', split_name, name + '.txt')
        
        # 이미지 복사
        shutil.copy2(src_img, dst_img)
        
        # 라벨 복사 (라벨이 존재하는 경우만)
        if os.path.exists(src_lbl):
            shutil.copy2(src_lbl, dst_lbl)

# 실행
copy_files(train_files, 'train')
copy_files(valid_files, 'valid')
copy_files(test_files, 'test')

print("-" * 30)
print(f"✅ 데이터셋 분할 완료! 저장 경로: {output_root}")

총 파일 수: 368
Train: 257 / Valid: 55 / Test: 56
------------------------------
📂 train 데이터 복사 중...
📂 valid 데이터 복사 중...
📂 test 데이터 복사 중...
------------------------------
✅ 데이터셋 분할 완료! 저장 경로: C:\Dev\KorailWheel\data\split_dataset


In [2]:
import os
import shutil
import random
from pathlib import Path
from tqdm import tqdm  # 진행상황 표시용 (없으면 pip install tqdm)

# ==========================================
# 1. 설정 (경로 확인 필수)
# ==========================================
BASE_DIR = Path("C:/Dev/KorailWheel/data")  # 프로젝트 루트 경로
SOURCE_DIR = BASE_DIR / "total"        # 원본 소스 폴더
DEST_DIR = BASE_DIR / "data"           # 분할된 데이터가 저장될 폴더

# 시드 고정 (재현성을 위해)
random.seed(42)

# ==========================================
# 2. 파일 목록 읽기 및 분류
# ==========================================
image_dir = SOURCE_DIR / "images"
label_dir = SOURCE_DIR / "labels"

# 이미지 파일 확장자 목록
valid_extensions = {".jpg", ".jpeg", ".png", ".bmp"}

# 전체 이미지 리스트 가져오기
all_images = [
    f for f in image_dir.iterdir() 
    if f.suffix.lower() in valid_extensions and f.is_file()
]

print(f"총 이미지 개수: {len(all_images)}장")

# 그룹 분류
# 'img'로 시작하는 파일 (고품질/오토라벨 보정본 추정) -> Valid/Test 후보
candidates = [img for img in all_images if img.name.lower().startswith("img")]

# 그 외 파일 (수동 라벨링 등) -> 무조건 Train
others = [img for img in all_images if not img.name.lower().startswith("img")]

print(f"- 'img'로 시작하는 파일(후보군): {len(candidates)}장")
print(f"- 그 외 파일(Train 확정): {len(others)}장")

# ==========================================
# 3. 데이터 분할 (Split)
# ==========================================
# 후보군이 최소 100장은 넘어야 함 (Valid 50 + Test 50)
if len(candidates) < 100:
    raise ValueError(f"오류: 'img'로 시작하는 파일이 100장 미만입니다. ({len(candidates)}장)")

# 랜덤 셔플
random.shuffle(candidates)

# 50장씩 뽑기
valid_set = candidates[:50]
test_set = candidates[50:100]

# 나머지는 모두 Train으로
train_from_candidates = candidates[100:]
train_set = train_from_candidates + others

print("\n[데이터셋 분할 결과]")
print(f"- Train : {len(train_set)}장 (나머지 img파일 + 그 외 파일)")
print(f"- Valid : {len(valid_set)}장 (img파일 중 랜덤 50장)")
print(f"- Test  : {len(test_set)}장 (img파일 중 랜덤 50장)")

# ==========================================
# 4. 파일 복사 실행
# ==========================================
def copy_files(file_list, split_name):
    # 저장할 경로 생성 (예: data/train/images)
    save_img_dir = DEST_DIR / split_name / "images"
    save_lbl_dir = DEST_DIR / split_name / "labels"
    
    save_img_dir.mkdir(parents=True, exist_ok=True)
    save_lbl_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"\nProcessing {split_name} data...")
    
    for img_path in tqdm(file_list):
        # 1. 이미지 복사
        shutil.copy2(img_path, save_img_dir / img_path.name)
        
        # 2. 라벨 복사 (짝이 맞는 txt 파일 찾기)
        label_name = img_path.stem + ".txt"
        label_src = label_dir / label_name
        
        if label_src.exists():
            shutil.copy2(label_src, save_lbl_dir / label_name)
        else:
            # 라벨이 없는 경우 경고 (필요 시 주석 해제)
            # print(f"Warning: Label not found for {img_path.name}")
            pass

# 기존 data 폴더가 있다면 삭제 후 재생성 (깨끗한 상태 유지)
if DEST_DIR.exists():
    shutil.rmtree(DEST_DIR)
    print(f"\n기존 {DEST_DIR} 폴더를 삭제하고 새로 생성합니다.")

# 복사 수행
copy_files(train_set, "train")
copy_files(valid_set, "valid")
copy_files(test_set, "test")

print("\n✅ 모든 작업이 완료되었습니다! 'data' 폴더를 확인하세요.")

총 이미지 개수: 368장
- 'img'로 시작하는 파일(후보군): 268장
- 그 외 파일(Train 확정): 100장

[데이터셋 분할 결과]
- Train : 268장 (나머지 img파일 + 그 외 파일)
- Valid : 50장 (img파일 중 랜덤 50장)
- Test  : 50장 (img파일 중 랜덤 50장)

Processing train data...


100%|██████████| 268/268 [00:00<00:00, 272.84it/s]



Processing valid data...


100%|██████████| 50/50 [00:00<00:00, 268.89it/s]



Processing test data...


100%|██████████| 50/50 [00:00<00:00, 301.07it/s]


✅ 모든 작업이 완료되었습니다! 'data' 폴더를 확인하세요.
